In [1]:
# base imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Важная настройка для корректной настройки pipeline!
import sklearn
sklearn.set_config(transform_output="pandas")

# Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

# for model learning
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
import category_encoders as ce

# notebook settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
# train датасет
df_train = pd.read_csv('../train.csv')

# тест датасет
df_test = pd.read_csv('../test.csv')

# слепленный датасет
df = pd.concat([df_train, df_test], axis=0)

In [20]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,YearBuilt,YearRemodAdd,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,Foundation,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Total_Access_Score,OverallCategory,Roof_Total_Score,Total_Exter_Rate,TotalBsmtScore,Heating_total_score,Garage_total_score
0,1,60,RL,65.0,8450,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,2003,2003,VinylSd,VinylSd,BrkFace,196.0,PConc,GLQ,706.0,Unf,0.0,150.0,856.0,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0,4,Average or Below,4,7,8,10,6
1,2,20,RL,80.0,9600,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,1976,1976,MetalSd,MetalSd,NaN,0.0,CBlock,ALQ,978.0,Unf,0.0,284.0,1262.0,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0,4,Good,4,6,11,10,6
2,3,60,RL,68.0,11250,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,2001,2002,VinylSd,VinylSd,BrkFace,162.0,PConc,GLQ,486.0,Unf,0.0,434.0,920.0,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0,4,Average or Below,4,7,9,10,6
3,4,70,RL,60.0,9550,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,1915,1970,Wd Sdng,Wd Shng,NaN,0.0,BrkTil,ALQ,216.0,Unf,0.0,540.0,756.0,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0,4,Average or Below,4,6,8,9,6
4,5,60,RL,84.0,14260,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,2000,2000,VinylSd,VinylSd,BrkFace,350.0,PConc,GLQ,655.0,Unf,0.0,490.0,1145.0,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0,4,Average or Below,4,7,10,10,6


### 1. Варианты, какие фичи можно соединить по общему смыслу и сделать новые, удалив старые

#### 1.1 Попытка соединить фичи, описывающие доступ (подъезд) к дому

In [5]:
# Street: Type of road access to property

#        Grvl	Gravel	- гравийная улица, 0 баллов
#        Pave	Paved - асфальтированная улица, 2 балла

# Alley: Type of alley access to property

#        Grvl	Gravel - гравийный переулок, 1 балл
#        Pave	Paved - асфальтироварный переулок, 2 балла
#        NA 	No alley access - нет доступа к переулку, 0 баллов

# PavedDrive: Paved driveway

#        Y	Paved - асфальтированная подъездная дорога, 2 балла
#        P	Partial Pavement - частично асфальтированная, 1 балл
#        N	Dirt/Gravel - грунт/гравий, 0 баллов


# Присвоение числовых значений
street_mapping = {'Pave': 2, 'Grvl': 0}
alley_mapping = {'Pave': 2, 'Grvl': 1, 'NA': 0}
paved_drive_mapping = {'Y': 2, 'P': 1, 'N': 0}

df['Street'] = df['Street'].map(street_mapping)
df['Alley'] = df['Alley'].fillna('NA').map(alley_mapping)
df['PavedDrive'] = df['PavedDrive'].map(paved_drive_mapping)

# Создание нового признака
df['Total_Access_Score'] = df['Street'] + df['Alley'] + df['PavedDrive']

# Удаление старых признаков
df = df.drop(columns=['Street', 'Alley', 'PavedDrive'])


#### 1.2 Объединение колонок по общему рейтингу дома

In [9]:
# OverallQual: Rates the overall material and finish of the house

#        10	Very Excellent
#        9	Excellent
#        8	Very Good
#        7	Good
#        6	Above Average
#        5	Average
#        4	Below Average
#        3	Fair
#        2	Poor
#        1	Very Poor
	
# OverallCond: Rates the overall condition of the house

#        10	Very Excellent
#        9	Excellent
#        8	Very Good
#        7	Good
#        6	Above Average	
#        5	Average
#        4	Below Average	
#        3	Fair
#        2	Poor
#        1	Very Poor

def categorize(row):
    if row['OverallQual'] > 8 and row['OverallCond'] > 8:
        return 'Excellent'
    elif row['OverallQual'] > 5 and row['OverallCond'] > 5:
        return 'Good'
    else:
        return 'Average or Below'
    
df['OverallCategory'] = df.apply(categorize, axis=1)

# удалим объединенные фичи
df = df.drop(columns=['OverallQual', 'OverallCond'])

#### 1.3 Объединение информации по крыше в один признак

In [11]:
# RoofStyle: Type of roof

#        Flat	Flat
#        Gable	Gable
#        Gambrel	Gabrel (Barn)
#        Hip	Hip
#        Mansard	Mansard
#        Shed	Shed
		
# RoofMatl: Roof material

#        ClyTile	Clay or Tile
#        CompShg	Standard (Composite) Shingle
#        Membran	Membrane
#        Metal	Metal
#        Roll	Roll
#        Tar&Grv	Gravel & Tar
#        WdShake	Wood Shakes
#        WdShngl	Wood Shingles

# Определение маппинга для RoofStyle
roof_style_mapping = {
    'Flat': 'Flat',
    'Gable': 'Sloped',
    'Gambrel': 'Sloped',
    'Hip': 'Sloped',
    'Mansard': 'Sloped',
    'Shed': 'Flat'
}

# Определение маппинга для RoofMatl
roof_matl_mapping = {
    'ClyTile': 'Premium',
    'CompShg': 'Standard',
    'Membran': 'Premium',
    'Metal': 'Premium',
    'Roll': 'Economy',
    'Tar&Grv': 'Economy',
    'WdShake': 'Premium',
    'WdShngl': 'Premium'
}

# Применение маппинга
df['RoofStyle_Grouped'] = df['RoofStyle'].map(roof_style_mapping)
df['RoofMatl_Grouped'] = df['RoofMatl'].map(roof_matl_mapping)

# Присвоение баллов
roof_style_scores = {
    'Flat': 1,
    'Sloped': 2
}

roof_matl_scores = {
    'Economy': 1,
    'Standard': 2,
    'Premium': 3
}

# Применение баллов
df['RoofStyle_Score'] = df['RoofStyle_Grouped'].map(roof_style_scores)
df['RoofMatl_Score'] = df['RoofMatl_Grouped'].map(roof_matl_scores)

# Создание общего показателя
df['Roof_Total_Score'] = df['RoofStyle_Score'] + df['RoofMatl_Score']

# Удаление старых и промежуточных колонок
df = df.drop(columns=[
    'RoofStyle_Grouped',
    'RoofMatl_Grouped',
    'RoofStyle_Score',
    'RoofMatl_Score',
    'RoofStyle',
    'RoofMatl'
    ])

#### 1.4 Объединение параметров по внешей оценке дома

In [13]:
# ExterQual: Evaluates the quality of the material on the exterior 
		
#        Ex	Excellent
#        Gd	Good
#        TA	Average/Typical
#        Fa	Fair
#        Po	Poor
		
# ExterCond: Evaluates the present condition of the material on the exterior
		
#        Ex	Excellent
#        Gd	Good
#        TA	Average/Typical
#        Fa	Fair
#        Po	Poor

# Присвоение числовых значений
qual_mapping = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
condition_mapping = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}

# Применение числовой замены
df['ExterQual'] = df['ExterQual'].map(qual_mapping)
df['ExterCond'] = df['ExterCond'].map(condition_mapping)

# Создание общего суммарного признака
df['Total_Exter_Rate'] = df['ExterQual'] + df['ExterCond']

# Удаление старых колонок
df = df.drop(columns=['ExterQual', 'ExterCond'])

#### 1.5 Объединение качественных признаков по подвалу

In [15]:
# BsmtQual: Evaluates the height of the basement

#        Ex	Excellent (100+ inches)	
#        Gd	Good (90-99 inches)
#        TA	Typical (80-89 inches)
#        Fa	Fair (70-79 inches)
#        Po	Poor (<70 inches
#        NA	No Basement

# BsmtCond: Evaluates the general condition of the basement

#        Ex	Excellent
#        Gd	Good
#        TA	Typical - slight dampness allowed
#        Fa	Fair - dampness or some cracking or settling
#        Po	Poor - Severe cracking, settling, or wetness
#        NA	No Basement
       
# BsmtExposure: Refers to walkout or garden level walls

#        Gd	Good Exposure
#        Av	Average Exposure (split levels or foyers typically score average or above)	
#        Mn	Mimimum Exposure
#        No	No Exposure
#        NA	No Basement

# Присвоение числовых значений
bsmt_qual_mapping = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
bsmt_cond_mapping = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
bsmt_exposure_mapping = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0}

# Обработка пропущенных значений и маппинг
df['BsmtQual'] = df['BsmtQual'].fillna('NA').map(bsmt_qual_mapping)
df['BsmtCond'] = df['BsmtCond'].fillna('NA').map(bsmt_cond_mapping)
df['BsmtExposure'] = df['BsmtExposure'].fillna('NA').map(bsmt_exposure_mapping)

# Создание общего балла
df['TotalBsmtScore'] = df['BsmtQual'] + df['BsmtCond'] + df['BsmtExposure']

# Удаление объединенных фичей
df = df.drop(columns=['BsmtQual', 'BsmtCond', 'BsmtExposure'])


#### 1.6 Объединение фичей по отоплению

In [17]:
# Heating: Type of heating
		
#        Floor	Floor Furnace
#        GasA	Gas forced warm air furnace
#        GasW	Gas hot water or steam heat
#        Grav	Gravity furnace	
#        OthW	Hot water or steam heat other than gas
#        Wall	Wall furnace
		
# HeatingQC: Heating quality and condition

#        Ex	Excellent
#        Gd	Good
#        TA	Average/Typical
#        Fa	Fair
#        Po	Poor

# Присвоение числовых значений
heating_mapping = {
    'GasA': 5,
    'GasW': 4,
    'OthW': 3,
    'Grav': 2,
    'Floor': 2,
    'Wall': 1
}

heating_qc_mapping = {
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1
}

# Маппинг значений
df['Heating'] = df['Heating'].map(heating_mapping)
df['HeatingQC'] = df['HeatingQC'].map(heating_qc_mapping)

# Создание суммарной оценки отопления
df['Heating_total_score'] = df['Heating'] + df['HeatingQC']

# Удаление объединенных фичей
df = df.drop(columns=['Heating', 'HeatingQC'])

#### 1.7 Объединение признаков по гаражу

In [19]:
# GarageQual: Garage quality

#        Ex	Excellent
#        Gd	Good
#        TA	Typical/Average
#        Fa	Fair
#        Po	Poor
#        NA	No Garage
		
# GarageCond: Garage condition

#        Ex	Excellent
#        Gd	Good
#        TA	Typical/Average
#        Fa	Fair
#        Po	Poor
#        NA	No Garage

# Присвоение числовых значений
garage_mapping = {
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1,
    'NA': 0
}

# Заполненение пропущенных значений и маппинг
df['GarageQual'] = df['GarageQual'].fillna('NA').map(garage_mapping)
df['GarageCond'] = df['GarageCond'].fillna('NA').map(garage_mapping)

# Создание общего признака
df['Garage_total_score'] = df['GarageQual'] + df['GarageCond']

# Удаление объединенных фичей
df = df.drop(columns=['GarageQual', 'GarageCond'])

### 2. Заполнение NaN-ов

In [ ]:
# проверка на НаНы
pd.DataFrame(data={'NaN_count': df.isna().sum(), 'data_type':df.dtypes})

#### 2.1 Замена не настоящих пропущенных значений в типе данных object

In [27]:
# колонки, в которых НаНы не НаНы, а просто - нет!
not_nan_list = [
    'Alley',
    'BsmtQual',
    'BsmtCond',
    'BsmtExposure',
    'BsmtFinType1',
    'BsmtFinType2',
    'FireplaceQu',
    'GarageType',
    'GarageFinish',
    'GarageQual',
    'GarageCond',
    'PoolQC',
    'Fence',
    'MiscFeature',
    'MasVnrType'
]

# получим колонки, где тип данных object
list_of_obj_values = df.select_dtypes(include='object').columns.tolist()
# len(list_of_obj_values) # 30 из 72

# получим список колонок, где надо заменить НаНы на какое-то осмысленное отрицательное значение
nan_replace_list = [x for x in not_nan_list if x in list_of_obj_values]

# значение, которым мы заменяем
replace_value = 'Absence'

# преобразование по замене значений на осмысленное отрицание, но не NaN
df[nan_replace_list] = df[nan_replace_list].fillna(replace_value)

#### 2.2 Замена оставшихся пропущенных значений в типе object

In [41]:
# Получим список колонок с типом object, где еще остались пропущенные значения
columns_obj_with_nan = [col for col in df.select_dtypes(include='object').columns if df[col].isna().any()]

# Посмотрим, сколько пропущенных значений в этих столбцах
nan_counts = {col: df[col].isna().sum() for col in columns_obj_with_nan}

# Выводим результат
for col, count in nan_counts.items():
    print(f"Фича '{col}' имеет {count} реальных пропущенных значения.")

Фича 'MSZoning' имеет 4 реальных пропущенных значения.
Фича 'Utilities' имеет 2 реальных пропущенных значения.
Фича 'Exterior1st' имеет 1 реальных пропущенных значения.
Фича 'Exterior2nd' имеет 1 реальных пропущенных значения.
Фича 'Electrical' имеет 1 реальных пропущенных значения.
Фича 'KitchenQual' имеет 1 реальных пропущенных значения.
Фича 'Functional' имеет 2 реальных пропущенных значения.
Фича 'SaleType' имеет 1 реальных пропущенных значения.


Так как количество пропущенных значений мало относительно общего количества элементов выборки,  
принимаем решение заполнить просто наиболее часто встречаемым значением!

#### 2.3 Замена пропущенных значений в числовых типах данных

In [55]:
# сформируем список колонок числовых типов данных, в которых есть пропущенные значения
# за исключением таргета, его не трогаем

columns_numeric_with_nan = [
    col for col in df.select_dtypes(include='number').columns
    if df[col].isna().any() and col != 'SalePrice'
]

# Посмотрим, сколько пропущенных значений в этих столбцах
nan_counts = {col: df[col].isna().sum() for col in columns_numeric_with_nan}

# Выводим результат
for col, count in nan_counts.items():
    print(f"Фича '{col}' имеет {count} реальных пропущенных значения.")

# Посмотрев и проанализировав вручную, сформировал списки для замены пропущенных
# значений медианой или средним

median_num_replace = [
    'LotFrontage',
    'MasVnrArea',
    'GarageYrBlt',
    'GarageCars',
    'GarageArea'
]
mean_num_replace = [
    'BsmtFinSF1',
    'BsmtFinSF2',
    'BsmtUnfSF',
    'TotalBsmtSF',
    'BsmtFullBath',
    'BsmtHalfBath'
]

Фича 'LotFrontage' имеет 486 реальных пропущенных значения.
Фича 'MasVnrArea' имеет 23 реальных пропущенных значения.
Фича 'BsmtFinSF1' имеет 1 реальных пропущенных значения.
Фича 'BsmtFinSF2' имеет 1 реальных пропущенных значения.
Фича 'BsmtUnfSF' имеет 1 реальных пропущенных значения.
Фича 'TotalBsmtSF' имеет 1 реальных пропущенных значения.
Фича 'BsmtFullBath' имеет 2 реальных пропущенных значения.
Фича 'BsmtHalfBath' имеет 2 реальных пропущенных значения.
Фича 'GarageYrBlt' имеет 159 реальных пропущенных значения.
Фича 'GarageCars' имеет 1 реальных пропущенных значения.
Фича 'GarageArea' имеет 1 реальных пропущенных значения.


#### 2.4 Повторяем импьютор

In [57]:
# Удалим столбец Id, он нам не нужен для обучения или прогнозирования
columns_to_drop = ['Id']


my_imputer = ColumnTransformer(
    transformers = [
        ('dop_columns', 'drop', columns_to_drop),
        ('num_imputer1', SimpleImputer(strategy='mean'), mean_num_replace),
        ('num_imputer2', SimpleImputer(strategy='median'), median_num_replace),
        ('cat_imputer', SimpleImputer(strategy='most_frequent'), columns_obj_with_nan)
    ],
    verbose_feature_names_out = False,
    remainder = 'passthrough',
    force_int_remainder_cols=False
)

# учим и сразу же трансформируем на датасете
df = my_imputer.fit_transform(X=df)

### 3. Кодирование категориальных переменных

In [62]:
# получим список всех категориальных переменных
list_of_obj_values = df.select_dtypes(include='object').columns.tolist()

# получим количество уникальных значений для каждого типа категориальных переменных
unique_values_obj_count = {col: df[col].nunique(dropna=True) for col in list_of_obj_values}

# получим два списка для кодирования
# если количество уникальных значений меньше либо равно 3 = OneHotEncoder будем применять
# во всех остальных случаях = TargetEncoder
one_hot_columns = [key for key, value in unique_values_obj_count.items() if value <= 3]
target_columns = [key for key, value in unique_values_obj_count.items() if value > 3]

#### 3.1 Повторим энкодеры

In [65]:
# Для энкодинга понадобится X-фичи и y-таргет, разделим
X, y = df.drop('SalePrice', axis=1), df['SalePrice']

In [66]:
my_encoder = ColumnTransformer(
    transformers=[
        ('one_hot_encoding', OneHotEncoder(sparse_output=False), one_hot_columns),
        ('target_encoding', ce.TargetEncoder(), target_columns)
    ],
    verbose_feature_names_out=False,
    remainder='passthrough',
    force_int_remainder_cols=False
)

# сразу же обучим и трансформируем
X = my_encoder.fit_transform(X=X, y=y)

### 4. Нормирование переменных

In [76]:
# нормируем значения переменных, максимальное значение которых больше 1
columns_to_scale = [col for col in X if X[col].max() > 1]

my_scaler = ColumnTransformer(
    transformers=[
        ('standard_scale', StandardScaler(), columns_to_scale)
    ],
    verbose_feature_names_out=False,
    remainder='passthrough',
    force_int_remainder_cols=False
)

X = my_scaler.fit_transform(X)

### 5. Выгрузка подготовленных значений

In [78]:
# общая трейновая и тестовая выборка по заданию
total_data = pd.concat([X, y], axis=1)

In [80]:
# подготовленная трейновая выборка
train_prepared = total_data.iloc[:1460, :]

# сохраняем
train_prepared.to_csv('../train_prepared_2d.csv', index=False)

In [83]:
# подготовленная тестовая выборка
test_prepared = total_data.iloc[1460:, :].drop(columns=['SalePrice'])

# сохраняем
test_prepared.to_csv('../test_prepared_2d.csv', index=False)